In [2]:
import polars as pl
import numpy as np
import pandas as pd

In [4]:
df=pd.read_csv('1990_to_2026.csv')

C:\Users\Asus\AppData\Local\Temp\ipykernel_11176\1756684966.py:1: DtypeWarning: Columns (11,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('1990_to_2026.csv')


In [8]:
df=df.sort_values(by=['tourney_date'],ascending=True)

In [10]:
df['tourney_date'].is_monotonic_increasing

True

In [15]:
df.columns

Index(['Unnamed: 0', 'tourney_id', 'tourney_name', 'surface', 'draw_size',
       'tourney_level', 'indoor', 'tourney_date', 'match_num', 'winner_id',
       'winner_seed', 'winner_entry', 'winner_name', 'winner_hand',
       'winner_ht', 'winner_ioc', 'winner_age', 'winner_rank',
       'winner_rank_points', 'loser_id', 'loser_seed', 'loser_entry',
       'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc', 'loser_age',
       'loser_rank', 'loser_rank_points', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced'],
      dtype='object')

In [12]:
df.isna().sum()

Unnamed: 0                 0
tourney_id                 0
tourney_name               0
surface                  143
draw_size                364
tourney_level              0
indoor                 10020
tourney_date               0
match_num                489
winner_id                  0
winner_seed            68748
winner_entry          101683
winner_name                0
winner_hand              804
winner_ht               2547
winner_ioc                 4
winner_age               363
winner_rank             1191
winner_rank_points      3404
loser_id                   1
loser_seed             89575
loser_entry            93118
loser_name                 0
loser_hand              1919
loser_ht                4905
loser_ioc                 12
loser_age                875
loser_rank              2628
loser_rank_points       5081
score                      2
best_of                    0
round                      0
minutes                15429
w_ace                  13098
w_df          

In [13]:
cols_cmn=[
    'tourney_date','tourney_name','surface',
    'tourney_level','round','best_of','minutes'
]

In [14]:
winner_df=df[cols_cmn].copy()

In [34]:
winner_df['player']=df['winner_name']
winner_df['opponent']=df['loser_name']
winner_df['player_rank']=df['winner_rank']
winner_df['opp_rank']=df['loser_rank']
winner_df['player_age']=df['winner_age']
winner_df['opp_age']=df['loser_age']
winner_df['player_hand']=df['winner_hand']
winner_df['player_ht']=df['winner_ht']
winner_df['opp_hand']=df['loser_hand']
winner_df['opp_ht']=df['loser_ht']
winner_df['player_aces']=df['w_ace']
winner_df['player_df']=df['w_df']
winner_df['player_bp_saved_pct']=df['w_1stIn'] / df['w_svpt']
winner_df['opp_aces']=df['l_ace']
winner_df['opp_df']=df['l_df']
winner_df['win']=1

In [35]:
loser_df=df[cols_cmn].copy()

In [36]:
loser_df['player']=df['loser_name']
loser_df['opponent']=df['winner_name']
loser_df['player_rank']=df['loser_rank']
loser_df['opp_rank']=df['winner_rank']
loser_df['player_age']=df['loser_age']
loser_df['opp_age']=df['winner_age']
loser_df['player_hand']=df['loser_hand']
loser_df['opp_hand']=df['winner_hand']
loser_df['player_ht']=df['loser_ht']
loser_df['opp_ht']=df['winner_ht']
loser_df['player_aces']=df['l_ace']
loser_df['player_df']=df['l_df']
loser_df['player_bp_saved_pct']=df['l_1stIn'] / df['l_svpt']
loser_df['opp_aces']=df['w_ace']
loser_df['opp_df']=df['w_df']
loser_df['win']=0

In [37]:
player_matches=pd.concat([winner_df,loser_df],ignore_index=True)

In [38]:
player_matches.shape

(230486, 23)

In [39]:
player_matches=player_matches.sort_values(['player','tourney_date'])

In [40]:
player_matches.head()

,tourney_date,tourney_name,surface,tourney_level,round,best_of,minutes,player,opponent,player_rank,...,win,player_hand,player_ht,opp_hand,opp_ht,player_aces,player_df,player_bp_saved_pct,opp_aces,opp_df
229793,20250912,Davis Cup WG1 R1: CHI vs LUX,Clay,D,RR,3,148.0,Aaron Gil Garcia,Cristian Garin,1361.0,...,0,NaN,NaN,R,185.0,2.0,4.0,0.5625,2.0,0.0
95,19900108,Sydney-1,Hard,250,R32,3,NaN,Aaron Krickstein,Jan Gunnarsson,7.0,...,1,R,183.0,R,185.0,NaN,NaN,NaN,NaN,NaN
116,19900108,Sydney-1,Hard,250,QF,3,NaN,Aaron Krickstein,David Wheaton,7.0,...,1,R,183.0,R,193.0,NaN,NaN,NaN,NaN,NaN
121,19900108,Sydney-1,Hard,250,R16,3,NaN,Aaron Krickstein,Paul Annacone,7.0,...,1,R,183.0,R,185.0,NaN,NaN,NaN,NaN,NaN
115355,19900108,Sydney-1,Hard,250,SF,3,NaN,Aaron Krickstein,Yannick Noah,7.0,...,0,R,183.0,R,193.0,NaN,NaN,NaN,NaN,NaN


In [41]:
player_matches.isna().sum()

tourney_date               0
tourney_name               0
surface                  286
tourney_level              0
round                      0
best_of                    0
minutes                30858
player                     0
opponent                   0
player_rank             3819
opp_rank                3819
player_age              1238
opp_age                 1238
win                        0
player_hand             2723
player_ht               7452
opp_hand                2723
opp_ht                  7452
player_aces            26197
player_df              26197
player_bp_saved_pct    26252
opp_aces               26197
opp_df                 26197
dtype: int64

In [43]:
player_matches.to_csv('model_csv.csv')

In [30]:
df.columns

Index(['Unnamed: 0', 'tourney_id', 'tourney_name', 'surface', 'draw_size',
       'tourney_level', 'indoor', 'tourney_date', 'match_num', 'winner_id',
       'winner_seed', 'winner_entry', 'winner_name', 'winner_hand',
       'winner_ht', 'winner_ioc', 'winner_age', 'winner_rank',
       'winner_rank_points', 'loser_id', 'loser_seed', 'loser_entry',
       'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc', 'loser_age',
       'loser_rank', 'loser_rank_points', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced'],
      dtype='object')

In [ ]:
player_matches['player_rank'].fillna(999,inplace=True)
player_matches['opp_rank'].fillna(999,inplace=True)
player_matches['player_age'].fillna(player_matches['player_age'].median(),inplace=True)
player_matches['opp_age'].fillna(player_matches['player_age'].median(),inplace=True)
player_matches['player_ht'].fillna(player_matches.groupby('player_hand')['player_ht'].transform('median'),inplace=True)
player_matches['opp_ht'].fillna(player_matches.groupby('opp_hand')['opp_ht'].transform('median'),inplace=True)
player_matches.dropna(subset=['player_aces','player_df','opp_aces','opp_df'],inplace=True)
player_matches['surface'].fillna(player_matches['surface'].mode()[0],inplace=True)
